# Point Cloud-to-Simplicial Complex Lifting Tutorial

***
This notebook shows how to import a dataset, with the desired lifting, and how to run a neural network using the loaded data.

The notebook is divided into sections:

- [Loading the dataset](#loading-the-dataset) loads the config files for the data and the desired tranformation, createsa a dataset object and visualizes it.
- [Loading and applying the lifting](#loading-and-applying-the-lifting) defines a simple neural network to test that the lifting creates the expected incidence matrices.
- [Create and run a simplicial nn model](#create-and-run-a-simplicial-nn-model) simply runs a forward pass of the model to check that everything is working as expected.

***
***

Note that for simplicity the notebook is setup to use a simple graph. However, there is a set of available datasets that you can play with.

To switch to one of the available datasets, simply change the *dataset_name* variable in [Dataset config](#dataset-config) to one of the following names:

* cocitation_cora
* cocitation_citeseer
* cocitation_pubmed
* MUTAG
* NCI1
* NCI109
* PROTEINS_TU
* AQSOL
* ZINC
***

### Imports and utilities

In [60]:
%load_ext autoreload
%autoreload 2
from modules.data.load.loaders import PointCloudLoader
from modules.data.preprocess.preprocessor import PreProcessor
from modules.utils.utils import (
    describe_data,
    load_dataset_config,
    load_model_config,
    load_transform_config,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading the Dataset

Here we just need to spicify the name of the available dataset that we want to load. First, the dataset config is read from the corresponding yaml file (located at `/configs/datasets/` directory), and then the data is loaded via the implemented `Loaders`.


We can then access to the data through the `load()` method:

In [61]:
dataset_name = "toy_point_cloud"
dataset_config = load_dataset_config(dataset_name)
loader = PointCloudLoader(dataset_config)

dataset = loader.load()


Dataset configuration for toy_point_cloud:

{'data_domain': 'point_cloud',
 'data_type': 'toy_dataset',
 'data_name': 'toy_point_cloud',
 'data_dir': 'datasets/point_cloud/toy_dataset',
 'num_points': 8,
 'num_classes': 2,
 'num_features': 1,
 'task': 'classification',
 'loss_type': 'cross_entropy'}


Processing...
Done!


## Loading and Applying the Lifting

In this section we will instantiate the whitness lifting. This lifting constructs a simplicial complex from a point cloud, called the **Witness complex** using the `GUDHI` library [[1]](https://gudhi.inria.fr/doc/latest/group__witness__complex.html). In the current implementation the Witness complex is defined over a point cloud $W$, called *witnesses*. From this set, a subset $L$ of *landmarks* is formed by randomly choosing the points. 

Current implementation can *only* lift to a *weak Euclidian Witness complex*:
 - *weak* means that $\sigma \in L$ is witnessed by $w\in W$ if $\forall l\in \sigma, \forall l' \in L: d(w,l)\leq d(w,l')$
 - *euclidian* means that the set of witnesses and landmarks are in Euclidian space.



***
[[1]](https://gudhi.inria.fr/doc/latest/group__witness__complex.html) Kachanovich S. (2020). Witness complex. GUDHI User and Reference Manual.
***

For simplicial complexes creating a lifting involves creating a `SimplicialComplex` object from topomodelx and adding simplices to it using the method `add_simplices_from`. The `SimplicialComplex` class then takes care of creating all the needed matrices.

Similarly to before, we can specify the transformation we want to apply through its type and id --the correxponding config files located at `/configs/transforms`. 

Note that the *tranform_config* dictionary generated below can contain a sequence of tranforms if it is needed.

This can also be used to explore liftings from one topological domain to another, for example using two liftings it is possible to achieve a sequence such as: graph -> simplicial complex -> hypergraph. 

In [62]:
transform_type = "liftings"
# If the transform is a topological lifting, it should include both the type of the lifting and the identifier
transform_id = "pointcloud2simplicial/witness_lifting"

# Read yaml file
transform_config = {
    "lifting": load_transform_config(transform_type, transform_id)
    # other transforms (e.g. data manipulations, feature liftings) can be added here
}


Transform configuration for pointcloud2simplicial/witness_lifting:

{'transform_type': 'lifting',
 'transform_name': 'WitnessLifting',
 'feature_lifting': 'ProjectionSum'}


We than apply the transform via our `PreProcesor`:

In [63]:
lifted_dataset = PreProcessor(dataset, transform_config, loader.data_dir)
describe_data(lifted_dataset)

Processing...
Done!



Dataset only contains 1 sample:


 - The complex has 8 0-cells.
 - The 0-cells have features dimension 1
 - The complex has 12 1-cells.
 - The 1-cells have features dimension 1
 - The complex has 7 2-cells.
 - The 2-cells have features dimension 1



## Create and Run a Simplicial NN Model

In this section a simple model is created to test that the used lifting works as intended. In this case the model uses the `up_laplacian_1` and the `down_laplacian_1` so the lifting should make sure to add them to the data.

In [64]:
from modules.models.simplicial.san import SANModel

model_type = "simplicial"
model_id = "san"
model_config = load_model_config(model_type, model_id)

model = SANModel(model_config, dataset_config)


Model configuration for simplicial SAN:

{'in_channels': None,
 'hidden_channels': 32,
 'out_channels': None,
 'n_layers': 2,
 'n_filters': 2,
 'order_harmonic': 5,
 'epsilon_harmonic': 0.1}


In [65]:
y_hat = model(lifted_dataset.get(0))

If everything is correct the cell above should execute without errors. 